In [1]:
import torch
print("Torch Version: ", torch.__version__)
print("Torch Version Cuda: ", torch.version.cuda)
print("Torch Version cuDnn: ", torch.backends.cudnn.version())
print("CUDA Available: ", torch.cuda.is_available())
print("CUDA Device Name: ", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

# Verificar se CUDA está disponível para acelerar o processamento
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

Torch Version:  2.5.0+cu124
Torch Version Cuda:  12.4
Torch Version cuDnn:  90800
CUDA Available:  True
CUDA Device Name:  NVIDIA RTX A1000 6GB Laptop GPU
Usando dispositivo: cuda


# Installing unsloth

In [ ]:
# it was installed in our venv...
# pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

In [4]:
initial_model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name="unsloth/Llama-3.2-3B-bnb-4bit",
    model_name = "unsloth/Llama-3.2-1B-Instruct", # or choose "unsloth/Llama-3.2-3B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # device_map="auto"
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = initial_model

==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    NVIDIA RTX A1000 6GB Laptop GPU. Num GPUs = 1. Max memory: 5.681 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [8]:
# move model to CPU before delete
# model.to("cpu")
# delete reference to the model
# del model

# import gc and call garbage collect to free orphan objects
# import gc
# gc.collect()

# Free GPU memor
# torch.cuda.empty_cache()

In [5]:
initial_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
 

# Dataset TeleQnA

We will load json files that we generated and create dataset to fine-tunning the model.

## Questions Release 17 

In [ ]:
import json

# Path to the TeleQnA processed question in JSON file with only Rel 17...
rel17_questions_path = r"../files/rel17_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_questions_path, "r", encoding="utf-8") as file:
    rel17_questions = json.load(file)
print(len(rel17_questions))

# Path to the TeleQnA processed question in JSON file with only Rel 17 and 100 questions...
rel17_100_questions_path = r"../files/rel17_100_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_200_questions_path, "r", encoding="utf-8") as file:
    rel17_200_questions = json.load(file)
print(len(rel17_200_questions))

rel17_other_questions = [q for q in rel17_questions if q not in rel17_100_questions]
print(len(rel17_other_questions))

rel17_other_questions_length = 500
rel17_other_questions = rel17_other_questions[:rel17_other_questions_length]
rel17_other_questions[0]

## Questions without Rel 17 and 18

In [ ]:
# Path to the TeleQnA processed question in JSON file with questions without Rel 17 and 18...
no_rel_17_18_path_questions = r"../files/no_rel_17_18_questions.json"

# Load the TeleQnA data...
with open(no_rel_17_18_path_questions, "r", encoding="utf-8") as file:
    no_rel_17_18_questions = json.load(file)
print(len(no_rel_17_18_questions))

no_rel_17_18_questions_length = 3500
no_rel_17_18_questions = no_rel_17_18_questions[:questions_no_rel_17_18_length]
print(len(no_rel_17_18_questions))
no_rel_17_18_questions[0]

## Train data

In [ ]:
train_questions = rel17_other_questions + no_rel_17_18_questions
print(len(train_questions))
train_questions[0]

We create two datasets, one with no options and half of questions and another wit options and the other half.

In [ ]:
from datasets import Dataset

# Structure to store pairs of questions and explanations
data = []

half_questions = len(train_questions)//2

# Fill the dataset with (question, explanation) pairs
for item in train_questions[:half_questions]:

    human_value = (
        f"{item['question']}"
    )

    # Combine the answer and explanation
    gpt_value = (
        f"{item['explanation']}"
    )

    # Create a dictionary for each input pair
    pair = [
        {'from': 'human', 'value': human_value},  # For the question
        {'from': 'gpt', 'value': gpt_value}  # For the explanation
    ]

    data.append(pair)  # Add the pair to the dataset

data_no_options = data
print(data_no_options[0])

In [ ]:
from datasets import Dataset

# Structure to store pairs of questions, options, answers, and explanations
data = []

# Fill the dataset with (question + options, answer + explanation) pairs
for item in train_questions[half_questions:]:

    # Extract options
    options = [f"{key}: {value}" for key, value in item.items() if 'option' in key]
    # Combine the question and options
    human_value = (
        f"Question: {item['question']}\n"
        f"Options:\n" + "\n".join(options) + "\n"
    )

    # Combine the answer and explanation
    gpt_value = (
        f"Answer: {item['answer']}\n"
        f"Explanation: {item['explanation']}"
    )

    # Create a dictionary for each input pair
    pair = [
        {'from': 'human', 'value': human_value},  # Question with options
        {'from': 'gpt', 'value': gpt_value}       # Answer with explanation
    ]

    data.append(pair)  # Add the pair to the dataset

# Create the dataset using Hugging Face
data_options = data
print(data_options[0])

Then we join these datasets and shuffle randomly.

In [ ]:
import random

data_total = data_no_options + data_options
# Shuffle the combined data
random.shuffle(data_total)
print(len(data_total))

Convert the list of pairs into the appropriate format
Transform the data into a Dataset
Save dataset on the disk

In [ ]:
formatted_data = {'conversations': data_total}
dataset = Dataset.from_dict(formatted_data)

print(dataset)
print(dataset[0])

dataset.save_to_disk('../files/train_questions_dataset_4000_questions')

In the next time that we run our model it isn't necessary to recreate dataset, just load it from the disk

In [ ]:
from datasets import load_from_disk

dataset_path = '../files/train_questions_dataset_4000_questions'

dataset = load_from_disk(dataset_path)

print(len(dataset))
dataset[0]
dataset[5]

# Format TeleQnA Dataset for training

In [ ]:
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_sharegpt

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

dataset
dataset[0]['text']

# Training

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# if necessary run command below to delete model
# del model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 300,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[0]["labels"]])

# Show current memory stats

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Initial Loss

In [ ]:
trainer.train_dataset[0].items()

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Configura o collator e DataLoader
collator = DataCollatorForSeq2Seq(tokenizer)
loader = DataLoader(trainer.train_dataset,
                    batch_size=2,  # Tamanho do batch escolhido
                    collate_fn=collator,
                    num_workers=2)

# Variáveis para armazenar a loss total e o número de exemplos
total_loss = 0.0
num_batches = 0

# Coloca o modelo em modo de avaliação
model.eval()

# Desativa o cálculo de gradiente para economizar memória
with torch.no_grad():
    for batch in tqdm(loader, desc="Calculating initial loss"):
        # Move o batch para a GPU (se disponível)
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Acumula a loss
        total_loss += outputs.loss.item()
        num_batches += 1

# Calcula a loss média
average_loss = total_loss / num_batches
print(f"Initial mean loss: {average_loss}")

# Training Session

In [ ]:
trainer_stats = trainer.train()

# Show final memory and time stats

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

model

# Loss Pos training

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Configure the collator and DataLoader
collator = DataCollatorForSeq2Seq(tokenizer)
loader = DataLoader(
    trainer.train_dataset,
    batch_size=2,  # Chosen batch size
    collate_fn=collator,
    num_workers=2
)

# Variables to store the total loss and the number of valid batches
total_loss = 0.0
num_batches = 0

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation to save memory
with torch.no_grad():
    for batch in tqdm(loader, desc="Calculating loss post-training"):
        # Move the batch to GPU (if available)
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Check if the loss is NaN
        if not torch.isnan(outputs.loss):
            total_loss += outputs.loss.item()
            num_batches += 1
            # print(outputs.loss.item())

print(f"Number of valid batches: {num_batches}")

# Calculate the average loss
if num_batches > 0:
    average_loss = total_loss / num_batches
    print(f"Post-training mean loss: {average_loss}")
else:
    print("No valid batches found.")

# Inference

## Question with option

In [ ]:
dataset[5]['conversations'][0]
question = dataset[5]['conversations'][0]['value']

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    # {"role": "user", "content": "How much is 1+1?"},
    {"role": "user", "content": question},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

## Question with no option

In [ ]:
dataset[0]['conversations'][0]['value']
question = dataset[0]['conversations'][0]['value']

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    # {"role": "user", "content": "How much is 1+1?"},
    {"role": "user", "content": question},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

# Save model

In [ ]:
model.save_pretrained("../models/llama_3.2_1B_FT_lora_4000_questions", safe_serialization=False)
tokenizer.save_pretrained("../models/llama_3.2_1B_FT_lora_4000_questions")

## Save merged

In [ ]:
model.save_pretrained_merged("../models/llama_3.2_1B_FT_lora_4bits_4000_questions", tokenizer, save_method = "merged_4bit",)

# Load Model

In [ ]:
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model_path = "../models/llama_3.2_1B_FT_lora_4000_questions"
# model_path = "model_3.2_lora_4bits"

# Carregar o modelo e o tokenizador separadamente
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype = dtype,
    load_in_4bit=load_in_4bit
)